In [1]:
# Current directory
import os
os.chdir('F:/Work/Experiment/pLM4ACE/model')

### Stacking model

In [2]:
import numpy as np
import pandas as pd


features = pd.read_csv("fusion_features/Data/single/ESM.csv", index_col=0, header=None)
labels = pd.read_csv("fusion_features/Data/label.csv", index_col=False, header=None)

print(features.shape)
print(labels.shape)
print(np.count_nonzero(labels==0))
print(np.count_nonzero(labels==1))

feature = np.array(features)
label = np.array(labels)

(1020, 320)
(1020, 1)
394
626


In [3]:
import math
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import adam_v2
from keras.layers import Dense,Dropout
from sklearn.preprocessing import scale
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, Dense,AveragePooling1D
from sklearn.metrics import confusion_matrix, roc_curve, auc


# model
def CNN(input_dim, out_dim, learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(filters = 64, kernel_size = 3, padding = 'same', activation= 'relu'))
    model.add(AveragePooling1D(pool_size=2,strides=1,padding="SAME"))
    model.add(Conv1D(filters = 64, kernel_size =  3, padding = 'same', activation= 'relu'))
    model.add(AveragePooling1D(pool_size=2,strides=1,padding="SAME"))
    model.add(Flatten())
    model.add(Dense(int(32), activation = 'relu'))
    model.add(Dense(out_dim, activation = 'softmax',name="Dense_2"))
    model.compile(loss = 'binary_crossentropy', optimizer = adam_v2.Adam(learning_rate), metrics =['binary_accuracy'])
    return model


def categorical_probas_to_classes(p):
    return np.argmax(p, axis=1)


def to_categorical(y, nb_classes=None):
    y = np.array(y, dtype='int')
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    for i in range(len(y)):
        Y[i, y[i]] = 1
    return Y


In [4]:
# 定义要调节的参数
# LR
param_grid = {
    'penalty': ['l2', 'l1'],
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}
# RF
# param_grid = {
#     'n_estimators': [50, 100, 200],   # 森林中树的数量
#     'max_depth': [None, 10, 20, 30],  # 每棵树的最大深度
#     'min_samples_split': [2, 5, 10],  # 分裂节点所需的最小样本数
#     'min_samples_leaf': [1, 2, 4],    # 叶子节点的最小样本数
#     'bootstrap': [True, False]        # 是否使用 bootstrap 样本
# }
# XGBoost
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [3, 5, 7],
#     'subsample': [0.6, 0.8, 1.0]
# }
# SVM
# param_grid = {
#     'C': [0.1, 1, 10, 100],
#     'kernel': ['linear', 'rbf', 'poly'],
#     'gamma': ['scale', 'auto']
# }
# KNN
# param_grid = {
#     'n_neighbors': [3, 5, 7, 10],  # 邻居数量
#     'weights': ['uniform', 'distance'],  # 权重：统一权重或根据距离加权
#     'p': [1, 2]  # 距离度量：p=1 曼哈顿距离，p=2 欧氏距离
# }



In [5]:
import math
import numpy as np
import xgboost as xgb
from sklearn.svm import SVC  # 导入 SVM 模型
from tensorflow.keras.models import load_model
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  # 导入 KNN 模型
from sklearn.metrics import precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier  # 导入随机森林
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc


# Create a directory to save the model
model_GRU_dir = 'save_models/GRU/Independence'
model_CNN_dir = 'save_models/CNN/Independence'
model_CapsuleGAN_dir = 'save_models/CapsuleGAN/Independence'

y = label
out_dim=2

BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
AP=[]

# Prepare for stacking model
stacked_features = []
stacked_labels = []
stacked_features_train = []
stacked_labels_train = []

for i in range(10):
    # Loading model
    if i in (0, 1, 2, 3):
        [sample_num, input_dim] = np.shape(feature)
        X = np.reshape(feature, (-1,1,input_dim))
        X_train, X_ind_test, y_train, y_ind_test = train_test_split(X, y, test_size=0.2, random_state=1111)

        model_path = os.path.join(model_GRU_dir, f'ESM_{i}.h5')
        clf = load_model(model_path)
        print(f'Model for fold {i} loaded from {model_path}')
    elif i in (6, 7, 8):
        [sample_num, input_dim] = np.shape(feature)
        X = np.reshape(feature, (-1,1,input_dim))
        X_train, X_ind_test, y_train, y_ind_test = train_test_split(X, y, test_size=0.2, random_state=1111)

        model_path = os.path.join(model_CNN_dir, f'ESM_{i}.h5')
        clf = load_model(model_path)
        print(f'Model for fold {i} loaded from {model_path}')
    else:
        X = feature
        X_train, X_ind_test, y_train, y_ind_test = train_test_split(X, y, test_size=0.2, random_state=1111)

        model_path = os.path.join(model_CapsuleGAN_dir, f'ESM_{i}.h5')
        clf = load_model(model_path)
        print(f'Model for fold {i} loaded from {model_path}')

    y_score_train = clf.predict(X_train)
    y_score = clf.predict(X_ind_test)

    # Collect features for stacking
    if len(stacked_features) == 0:
        stacked_features_train = y_score_train
        stacked_labels_train = y_train
        stacked_features = y_score
        stacked_labels = y_ind_test
    else:
        stacked_features_train = np.hstack([stacked_features_train, y_score_train])
        stacked_features = np.hstack([stacked_features, y_score])


# [sample_num, input_dim] = np.shape(stacked_features_train)
# stacked_features_train = np.reshape(stacked_features_train, (-1,1,input_dim))
# stacked_features = np.reshape(stacked_features, (-1,1,input_dim))
# out_dim=2
# labels_train = to_categorical(y_train)

# 在fit函数之前，将y_ind_test转换为一维数组
stacked_labels_train = stacked_labels_train.ravel()
stacked_labels = stacked_labels.ravel()

# Train meta-learner (stacking classifier) with the collected features
meta_learner = LogisticRegression(max_iter=5000)        # 逻辑回归
# meta_learner = RandomForestClassifier()     # 随机森林
# meta_learner = xgb.XGBClassifier(eval_metric='logloss')   # XGBoost
# meta_learner = SVC(probability=True)
# meta_learner = KNeighborsClassifier()
meta_learner = GridSearchCV(meta_learner, param_grid, cv=5, scoring='balanced_accuracy')     # 最佳参数搜索
meta_learner.fit(stacked_features_train, stacked_labels_train)
# 输出最优参数
print("Best parameters: ", meta_learner.best_params_)

# meta_learner = CNN(input_dim, out_dim, 0.0005)      # CNN
# meta_learner.fit(stacked_features_train, labels_train, batch_size=64, epochs=60)

# Get final predictions from meta-learner
# pred_score = meta_learner.predict(stacked_features)
pred_score = meta_learner.predict_proba(stacked_features)
final_predictions = categorical_probas_to_classes(pred_score)

TP, FP, FN, TN = confusion_matrix(y_ind_test, final_predictions).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
Sn_collecton = TP/(TP+FN)
Sp_collecton = TN/(TN+FP)
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton = MCC
BACC_collecton = 0.5*TP/(TP+FN)+0.5*TN/(TN+FP)
# ROC curve
fpr, tpr, _ = roc_curve(y_ind_test, pred_score[:, 1])
auc_roc = auc(fpr, tpr)
AUC_collecton = auc_roc
# PR curve
precision, recall, _ = precision_recall_curve(y_ind_test, pred_score[:, 1])
average_precision = average_precision_score(y_ind_test, pred_score[:, 1])
AP = average_precision

# Output
results = [
    f"BACC: {round(BACC_collecton, 3)}",
    f"Sn: {round(Sn_collecton, 3)}",
    f"Sp: {round(Sp_collecton, 3)}",
    f"MCC: {round(MCC_collecton, 3)}",
    f"AUC: {round(AUC_collecton, 3)}",
    f"AP: {round(AP, 3)}"
]

for result in results:
    print(result)

# Append the results to the file
with open('result/results_Stack.txt', 'a') as file:
    file.write("----------------------------------------\n")
    for result in results:
        file.write(result + '\n')


Model for fold 0 loaded from save_models/GRU/Independence\ESM_0.h5
Model for fold 1 loaded from save_models/GRU/Independence\ESM_1.h5
Model for fold 2 loaded from save_models/GRU/Independence\ESM_2.h5
Model for fold 3 loaded from save_models/GRU/Independence\ESM_3.h5
Model for fold 4 loaded from save_models/CapsuleGAN/Independence\ESM_4.h5
Model for fold 5 loaded from save_models/CapsuleGAN/Independence\ESM_5.h5
Model for fold 6 loaded from save_models/CNN/Independence\ESM_6.h5
Model for fold 7 loaded from save_models/CNN/Independence\ESM_7.h5
Model for fold 8 loaded from save_models/CNN/Independence\ESM_8.h5
Model for fold 9 loaded from save_models/CapsuleGAN/Independence\ESM_9.h5
Best parameters:  {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
BACC: 0.919
Sn: 0.971
Sp: 0.866
MCC: 0.805
AUC: 0.964
AP: 0.974


In [6]:
# 保存ROC曲线相关参数
np.savez('graph/ACE/ROC/Stack.npz', fpr=fpr, tpr=tpr, roc_auc=AUC_collecton)

# 保存PR曲线相关参数
np.savez('graph/ACE/PR/Stack.npz', recall=recall, precision=precision, average_precision=AP)